In [1]:
# Imports
import pandas as pd
import numpy as np
from google.cloud import bigquery
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import pickle
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliothèques importées")

✅ Bibliothèques importées


In [2]:
# Connexion BigQuery
project_id = 'students-group1'
client = bigquery.Client(project=project_id)

print("Chargement des données NETTOYÉES depuis BigQuery...")

# Charger depuis BigQuery (données nettoyées)
query_movies = """
SELECT movieId, title, genres 
FROM `students-group1.group1_movie_analysis.movies_clean`
"""
df_movies = client.query(query_movies).to_dataframe()

query_ratings = """
SELECT * 
FROM `students-group1.group1_movie_analysis.ratings_clean`
"""
df_ratings = client.query(query_ratings).to_dataframe()

# Recréer la colonne genres_list
df_movies['genres_list'] = df_movies['genres'].apply(
    lambda x: x.split('|') if pd.notna(x) else []
)

print(f"✅ {len(df_movies)} films nettoyés chargés depuis BigQuery")
print(f"✅ {len(df_ratings)} ratings nettoyés chargés depuis BigQuery")
print(f"✅ {df_ratings['userId'].nunique()} utilisateurs")

Chargement des données NETTOYÉES depuis BigQuery...
✅ 3855 films nettoyés chargés depuis BigQuery
✅ 94121 ratings nettoyés chargés depuis BigQuery
✅ 668 utilisateurs


In [3]:
# Nettoyer les genres
def extract_genres(genres_str):
    if pd.isna(genres_str) or genres_str == "(no genres listed)":
        return []
    return genres_str.split('|')

df_movies['genres_list'] = df_movies['genres'].apply(extract_genres)

# Créer une matrice des genres (one-hot encoding)
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
genres_matrix = mlb.fit_transform(df_movies['genres_list'])
genres_df = pd.DataFrame(genres_matrix, columns=mlb.classes_, index=df_movies['movieId'])

print(f"✅ {len(mlb.classes_)} genres uniques détectés")
print(f"Genres : {list(mlb.classes_)[:10]}...")  # Afficher les 10 premiers

✅ 19 genres uniques détectés
Genres : ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir']...


In [4]:
# Créer la matrice utilisateur-film (pivot table)
user_movie_matrix = df_ratings.pivot_table(
    index='userId',
    columns='movieId',
    values='rating',
    fill_value=0
)

print(f"✅ Matrice créée : {user_movie_matrix.shape}")
print(f"   - {user_movie_matrix.shape[0]} utilisateurs")
print(f"   - {user_movie_matrix.shape[1]} films")
print(f"   - Densité : {(user_movie_matrix > 0).sum().sum() / (user_movie_matrix.shape[0] * user_movie_matrix.shape[1]) * 100:.2f}%")

✅ Matrice créée : (668, 3855)
   - 668 utilisateurs
   - 3855 films
   - Densité : 3.65%


Fonction de recommandation pour NOUVEAUX utilisateurs (Content-Based)
fonction qui  recommande des films populaires dans les genres préférés.

In [5]:
def recommend_for_new_user(preferred_genres=None, n_recommendations=10):
    """
    Recommande des films pour un nouvel utilisateur basé sur les genres
    
    Args:
        preferred_genres: Liste de genres préférés (ex: ['Action', 'Comedy'])
        n_recommendations: Nombre de films à recommander
    
    Returns:
        DataFrame avec les recommandations
    """
    # Calculer la popularité des films (nombre de notes)
    movie_popularity = df_ratings.groupby('movieId').agg({
        'rating': ['count', 'mean']
    }).reset_index()
    movie_popularity.columns = ['movieId', 'num_ratings', 'avg_rating']
    
    # Fusionner avec les infos des films
    recommendations = df_movies.merge(movie_popularity, on='movieId', how='left')
    recommendations['num_ratings'] = recommendations['num_ratings'].fillna(0)
    recommendations['avg_rating'] = recommendations['avg_rating'].fillna(0)
    
    # Filtrer par genres si spécifié
    if preferred_genres:
        mask = recommendations['genres_list'].apply(
            lambda x: any(genre in x for genre in preferred_genres)
        )
        recommendations = recommendations[mask]
    
    # Filtrer : au moins 20 notes pour être considéré
    recommendations = recommendations[recommendations['num_ratings'] >= 20]
    
    # Calculer un score combiné (popularité + note)
    recommendations['score'] = (
        recommendations['num_ratings'] * 0.3 + 
        recommendations['avg_rating'] * 100 * 0.7
    )
    
    # Trier et retourner les top N
    recommendations = recommendations.sort_values('score', ascending=False)
    
    return recommendations[['movieId', 'title', 'genres', 'num_ratings', 'avg_rating', 'score']].head(n_recommendations)

# Test
print("TEST : Recommandations pour un nouvel utilisateur qui aime l'Action et la Science-Fiction")
print("="*80)
new_user_recs = recommend_for_new_user(preferred_genres=['Action', 'Sci-Fi'], n_recommendations=10)
print(new_user_recs.to_string(index=False))

TEST : Recommandations pour un nouvel utilisateur qui aime l'Action et la Science-Fiction
 movieId                                                                          title                                  genres  num_ratings  avg_rating      score
    2571                                                             Matrix, The (1999)                  Action|Sci-Fi|Thriller          261    4.264368 376.805747
     260                                      Star Wars: Episode IV - A New Hope (1977)                 Action|Adventure|Sci-Fi          273    4.188645 375.105128
    1196                          Star Wars: Episode V - The Empire Strikes Back (1980)                 Action|Adventure|Sci-Fi          228    4.228070 364.364912
    1198 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)                        Action|Adventure          224    4.212054 362.043750
     110                                                              Braveheart (1995)   

Fonction de recommandation basée sur la similarité (Collaborative Filtering)
Cette fonction trouve des utilisateurs similaires et recommande leurs films préférés.

In [6]:
def recommend_based_on_ratings(user_ratings, n_recommendations=10, n_similar_users=20):
    """
    Recommande des films basé sur les notes données par un utilisateur
    
    Args:
        user_ratings: Dict {movieId: rating} - les notes de l'utilisateur
        n_recommendations: Nombre de films à recommander
        n_similar_users: Nombre d'utilisateurs similaires à considérer
    
    Returns:
        DataFrame avec les recommandations
    """
    # Créer un vecteur pour le nouvel utilisateur
    new_user_vector = pd.Series(0, index=user_movie_matrix.columns)
    
    for movie_id, rating in user_ratings.items():
        if movie_id in new_user_vector.index:
            new_user_vector[movie_id] = rating
    
    # Calculer la similarité avec tous les utilisateurs existants
    similarities = cosine_similarity(
        [new_user_vector.values],
        user_movie_matrix.values
    )[0]
    
    # Trouver les utilisateurs les plus similaires
    similar_users_idx = np.argsort(similarities)[::-1][:n_similar_users]
    similar_users = user_movie_matrix.iloc[similar_users_idx]
    
    # Calculer les scores de recommandation (moyenne pondérée)
    weights = similarities[similar_users_idx]
    weighted_ratings = similar_users.T.dot(weights) / weights.sum()
    
    # Exclure les films déjà notés par l'utilisateur
    already_rated = list(user_ratings.keys())
    weighted_ratings = weighted_ratings.drop(already_rated, errors='ignore')
    
    # Trier et obtenir les top N
    top_movies = weighted_ratings.sort_values(ascending=False).head(n_recommendations)
    
    # Ajouter les infos des films
    recommendations = df_movies[df_movies['movieId'].isin(top_movies.index)].copy()
    recommendations['predicted_rating'] = recommendations['movieId'].map(top_movies)
    
    # Ajouter la popularité
    movie_stats = df_ratings.groupby('movieId').agg({
        'rating': ['count', 'mean']
    }).reset_index()
    movie_stats.columns = ['movieId', 'num_ratings', 'avg_rating']
    
    recommendations = recommendations.merge(movie_stats, on='movieId', how='left')
    recommendations = recommendations.sort_values('predicted_rating', ascending=False)
    
    return recommendations[['movieId', 'title', 'genres', 'predicted_rating', 'num_ratings', 'avg_rating']]

# Test : simuler un utilisateur qui a noté quelques films
print("\nTEST : Recommandations basées sur les notes d'un utilisateur")
print("="*80)
print("L'utilisateur a aimé :")
test_ratings = {
    296: 5.0,   # Pulp Fiction
    318: 5.0,   # Shawshank Redemption
    593: 4.5,   # Silence of the Lambs
    260: 4.0    # Star Wars
}

for movie_id, rating in test_ratings.items():
    movie_title = df_movies[df_movies['movieId'] == movie_id]['title'].values[0]
    print(f"  - {movie_title} : {rating}/5")

print("\nRecommandations pour cet utilisateur :")
print("-"*80)
collaborative_recs = recommend_based_on_ratings(test_ratings, n_recommendations=10)
print(collaborative_recs.to_string(index=False))


TEST : Recommandations basées sur les notes d'un utilisateur
L'utilisateur a aimé :
  - Pulp Fiction (1994) : 5.0/5
  - Shawshank Redemption, The (1994) : 5.0/5
  - Silence of the Lambs, The (1991) : 4.5/5
  - Star Wars: Episode IV - A New Hope (1977) : 4.0/5

Recommandations pour cet utilisateur :
--------------------------------------------------------------------------------
 movieId                                                 title                                      genres  predicted_rating  num_ratings  avg_rating
     356                                   Forrest Gump (1994)                    Comedy|Drama|Romance|War          3.216459          311    4.138264
     457                                  Fugitive, The (1993)                                    Thriller          2.474029          244    3.930328
     590                             Dances with Wolves (1990)                     Adventure|Drama|Western          2.140494          201    3.766169
     150          

Fonction hybride intelligente
Cette fonction combine les deux approches selon la situation.

In [7]:
def smart_recommend(user_ratings=None, preferred_genres=None, n_recommendations=10):
    """
    Système de recommandation hybride intelligent
    
    Args:
        user_ratings: Dict {movieId: rating} ou None pour un nouvel utilisateur
        preferred_genres: Liste de genres préférés
        n_recommendations: Nombre de films à recommander
    
    Returns:
        DataFrame avec les recommandations
    """
    # Cas 1 : Nouvel utilisateur sans notes
    if user_ratings is None or len(user_ratings) == 0:
        print("🆕 Nouvel utilisateur détecté - Recommandations basées sur les genres populaires")
        return recommend_for_new_user(preferred_genres, n_recommendations)
    
    # Cas 2 : Peu de notes (< 5) - Mélange content + collaborative
    elif len(user_ratings) < 5:
        print(f"📊 {len(user_ratings)} note(s) disponible(s) - Recommandations hybrides")
        
        # 50% collaborative, 50% content-based
        n_collab = n_recommendations // 2
        n_content = n_recommendations - n_collab
        
        collab_recs = recommend_based_on_ratings(user_ratings, n_collab)
        content_recs = recommend_for_new_user(preferred_genres, n_content)
        
        # Combiner les deux
        collab_recs['source'] = 'Similarité utilisateurs'
        content_recs['source'] = 'Popularité + Genres'
        
        return pd.concat([collab_recs, content_recs]).head(n_recommendations)
    
    # Cas 3 : Beaucoup de notes (>= 5) - Principalement collaborative
    else:
        print(f"⭐ {len(user_ratings)} notes disponibles - Recommandations personnalisées")
        return recommend_based_on_ratings(user_ratings, n_recommendations)

# Test des 3 cas
print("="*80)
print("SCÉNARIO 1 : Nouvel utilisateur sans historique")
print("="*80)
recs1 = smart_recommend(preferred_genres=['Action', 'Thriller'], n_recommendations=5)
print(recs1[['title', 'genres', 'avg_rating']].to_string(index=False))

print("\n" + "="*80)
print("SCÉNARIO 2 : Utilisateur avec 2 notes")
print("="*80)
recs2 = smart_recommend(user_ratings={296: 5.0, 318: 4.5}, preferred_genres=['Drama'], n_recommendations=5)
print(recs2[['title', 'genres']].to_string(index=False))

print("\n" + "="*80)
print("SCÉNARIO 3 : Utilisateur avec 10 notes")
print("="*80)
many_ratings = {296: 5.0, 318: 5.0, 593: 4.5, 260: 4.0, 480: 4.5, 
                527: 4.0, 589: 3.5, 1198: 4.5, 2571: 5.0, 356: 4.0}
recs3 = smart_recommend(user_ratings=many_ratings, n_recommendations=5)
print(recs3[['title', 'genres', 'predicted_rating']].to_string(index=False))

SCÉNARIO 1 : Nouvel utilisateur sans historique
🆕 Nouvel utilisateur détecté - Recommandations basées sur les genres populaires
                                    title                      genres  avg_rating
                      Pulp Fiction (1994) Comedy|Crime|Drama|Thriller    4.160000
         Silence of the Lambs, The (1991)       Crime|Horror|Thriller    4.194828
                       Matrix, The (1999)      Action|Sci-Fi|Thriller    4.264368
Star Wars: Episode IV - A New Hope (1977)     Action|Adventure|Sci-Fi    4.188645
               Usual Suspects, The (1995)      Crime|Mystery|Thriller    4.328947

SCÉNARIO 2 : Utilisateur avec 2 notes
📊 2 note(s) disponible(s) - Recommandations hybrides
                           title                      genres
Silence of the Lambs, The (1991)       Crime|Horror|Thriller
             Forrest Gump (1994)    Comedy|Drama|Romance|War
Shawshank Redemption, The (1994)                 Crime|Drama
             Pulp Fiction (1994) Comedy|Crim

Sauvegarder le modèle
On va sauvegarder tous les éléments nécessaires pour pouvoir les réutiliser dans l'API.

In [9]:
import pickle
import os

# Créer le dossier models s'il n'existe pas
os.makedirs('../models', exist_ok=True)

print("Sauvegarde du modèle et des données...")

# Sauvegarder les DataFrames NETTOYÉS
df_movies.to_pickle('../models/df_movies_clean.pkl')
df_ratings.to_pickle('../models/df_ratings_clean.pkl')

# Sauvegarder la matrice utilisateur-film
user_movie_matrix.to_pickle('../models/user_movie_matrix.pkl')

# Sauvegarder la matrice des genres
genres_df.to_pickle('../models/genres_df.pkl')

# Sauvegarder le MultiLabelBinarizer
with open('../models/mlb.pkl', 'wb') as f:
    pickle.dump(mlb, f)

print("✅ Modèle sauvegardé dans le dossier 'models/'")
print("Fichiers créés :")
print("  - df_movies_clean.pkl")      # ← Nom corrigé
print("  - df_ratings_clean.pkl")     # ← Nom corrigé
print("  - user_movie_matrix.pkl")
print("  - genres_df.pkl")
print("  - mlb.pkl")

Sauvegarde du modèle et des données...
✅ Modèle sauvegardé dans le dossier 'models/'
Fichiers créés :
  - df_movies_clean.pkl
  - df_ratings_clean.pkl
  - user_movie_matrix.pkl
  - genres_df.pkl
  - mlb.pkl
